# Experiment: Working with Dolly
## Last Updated $DATE -  $AUTHOR.

```
Summary of High Level Research Question
```

Try to scope your experiments such you can answer your research question in 1-3 hours.
This is an ideal time block to enter flow / deep work, but short enough that you will still feel 
motivated by a relatively tight feedback loop.

If a problem seems like it needs more time that that, 

### High Level Experiment Design

## Goals:
```
List of specific goals that this experiment seeks to achieve.

This should fall under a few categories:
- Development of Intuition about a _specific_ topic
- Novel Research or Insight that could lead to a publishable result
- Meaningfully explore a topic which could lead to an improvement in product

Guiding principles should understanding, insight, and value creation.
```

## Tasks & Experiment Design

```
A list of specific tasks that are going to be tested 

```


## Outcomes

```
Document high level research findings and how
```


In [1]:
# Install things into ENV
# TODO: Setup up a container and push to docker that contains all these
%pip install git+https://github.com/neelnanda-io/TransformerLens.git
%pip install circuitsvis
%pip install plotly


  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-06gql0_c
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-06gql0_c
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 0ffcc8ad647d9e991f4c2596557a9d7475617773
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 14.3 MB/s eta 0:00:00
  Created wheel for transformer-lens: filename=transformer_lens-0.0.0-py3-none-any.whl size=89099 sha256=e9c4c1a436abe06007462feb506e9c233fd924e1ed72f98b6e58c637a50413aa
  Stored in directory:

In [2]:
# Generic Set of Imports for MI Research
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
from pprint import pprint
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [3]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [4]:
# Setup PyTorch configuration for inference based experiments
# NOTE: Mark as False if you want to do any kind of training 
#       as part of your experimentation

INFERENCE_ONLY_EXPERIMENT = True
if INFERENCE_ONLY_EXPERIMENT:
    torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

import plotly.io as pio
pio.renderers.default = "notebook_connected"

cuda


In [5]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [6]:
# Load Circuit Visualizations
# TODO: Explore building out our own packages / tooling
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Vivek")


In [7]:
# Load & Run a Model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-7b")
hf_model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-7b")

print("Loaded hf_model, hooking transformer into TransformerLens!")
# model = HookedTransformer.from_pretrained(
#     "EleutherAI/pythia-6.9b-deduped",
#     center_unembed=False,
#     center_writing_weights=False,
#     fold_ln=False,
#     refactor_factored_attn_matrices=True,
#     hf_model=hf_model
# )

### Janky Shit
### TODO: Figure out how this library actually works and make this a cleaner integration.
import transformer_lens.loading_from_pretrained as loading
# Get the model name used in HuggingFace, rather than the alias.
official_model_name = loading.get_official_model_name("EleutherAI/pythia-6.9b-deduped")


# Load the config into an HookedTransformerConfig object. If loading from a
# checkpoint, the config object will contain the information about the
# checkpoint
cfg = loading.get_pretrained_model_config(
    official_model_name,
    checkpoint_index=None,
    checkpoint_value=None,
    fold_ln=False,
    device=device,
    n_devices=1,
)
print(cfg)
cfg.d_vocab = 50280
cfg.d_vocab_out = 50280
print(cfg)


# Get the state dict of the model (ie a mapping of parameter names to tensors), processed to match the HookedTransformer parameter names.
state_dict = loading.get_pretrained_state_dict(
    official_model_name, cfg, hf_model
)

# Create the HookedTransformer object
model = HookedTransformer(cfg, tokenizer=tokenizer)

model.load_and_process_state_dict(
    state_dict,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    refactor_factored_attn_matrices=False,
    move_state_dict_to_device=True,
)

print(f"Loaded pretrained model into HookedTransformer!")

model_description_text = """For this demo notebook we'll look at Dolly v2. It is based on pythia 6.9b, but we use the weights for dolly v2. To try the model the model out, let's find the loss on this paragraph!"""
# return_type of model can be loss, logits, both, or none!
loss = model(model_description_text, return_type="loss")
print("Model loss:", loss)


Loaded hf_model, hooking transformer into TransformerLens!


HookedTransformerConfig:
{'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 128,
 'd_mlp': 16384,
 'd_model': 4096,
 'd_vocab': 50432,
 'd_vocab_out': 50432,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.0125,
 'model_name': 'pythia-6.9b-deduped',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 32,
 'n_layers': 32,
 'n_params': 6442450944,
 'normalization_type': 'LN',
 'original_architecture': 'GPTNeoXForCausalLM',
 'parallel_attn_mlp': True,
 'positional_embedding_type': 'rotary',
 'rotary_dim': 32,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'EleutherAI/pythia-6.9b-deduped',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_tokens': False,
 'use_local_attn': False,
 'use_split_qkv_inp

In [8]:
# DOLLY V2 - 7B Config
pprint(model.cfg)

# Transformer Lens Note:
# get_token_position, to_tokens, to_string, to_str_tokens, prepend_bos, to_single_token
# are all methods that are added to the model object by TransformerLens

HookedTransformerConfig:
{'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 128,
 'd_mlp': 16384,
 'd_model': 4096,
 'd_vocab': 50280,
 'd_vocab_out': 50280,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.0125,
 'model_name': 'pythia-6.9b-deduped',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 32,
 'n_layers': 32,
 'n_params': 6442450944,
 'normalization_type': 'LN',
 'original_architecture': 'GPTNeoXForCausalLM',
 'parallel_attn_mlp': True,
 'positional_embedding_type': 'rotary',
 'rotary_dim': 32,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'EleutherAI/pythia-6.9b-deduped',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_tokens': False,
 'use_local_attn': False,
 'use_split_qkv_inp

In [35]:

%pip install git+https://github.com/neelnanda-io/Easy-Transformer.git@clean-transformer-demo
  # Install another version of node that makes PySvelte work way faster
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  
%pip install git+https://github.com/neelnanda-io/PySvelte.git
%pip install fancy_einsum
%pip install einops
%pip install datasets
%pip install transformers
import datasets
import transformers
import plotly.express as px
import einops
from fancy_einsum import einsum
from dataclasses import dataclass

import torch
import torch.nn as nn
import numpy as np
import math
from easy_transformer import EasyTransformer
from easy_transformer.utils import get_corner, gelu_new, tokenize_and_concatenate
import tqdm.auto as tqdm
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/neelnanda-io/Easy-Transformer.git (to revision clean-transformer-demo) to /tmp/pip-req-build-gdr7azw_
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/Easy-Transformer.git /tmp/pip-req-build-gdr7azw_
  Running command git checkout -b clean-transformer-demo --track origin/clean-transformer-demo
  Switched to a new branch 'clean-transformer-demo'
  Branch 'clean-transformer-demo' set up to track remote branch 'clean-transformer-demo' from 'origin'.
  Resolved https://github.com/neelnanda-io/Easy-Transformer.git to commit 1f25219e631aeb478d17075d47274db32c874e88
  Preparing metadata (setup.py) ... done
No

ModuleNotFoundError: No module named 'easy_transformer.utils'

In [31]:
#from easy_transformer.utils import get_corner, gelu_new, tokenize_and_concatenate


ImportError: cannot import name 'get_corner' from 'easy_transformer' (/notebooks/easy_transformer/__init__.py)

In [34]:
sample_string = "On halloween, all the children go Trick or"

print(model.to_str_tokens(sample_string)) # Shows tokenization split
print(model.to_tokens(sample_string)) #converts string to integer labeled tokens and then returns a tensor on models device of shape (batch, position)
# NOTE: in GPT2, 50256 is the token for EOS, BOS, and Padding.
# To single token converts string to a single integer, useful for looking up logits
# to_string converts a tensor of tokens to a string


#model.blocks.register_forward_hook  



model.generate(sample_string,
               temperature=0,
               max_new_tokens=1)



from pprint import pprint
batch_size = 8
num_epochs = 1
max_steps = 1
log_every = 1
lr = 1e-3
weight_decay = 1e-2
overfitMax=1
#model_cfg = Config(debug=False, d_model=256, n_heads=4, d_head=64, d_mlp=1024, n_layers=2, n_ctx=256, d_vocab=reference_gpt2.cfg.d_vocab)


optimizer_copy = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)


print("done one")
losses = []

#print(dataset)
#print(dataset[0]['text'][:100])
from EasyTransformer import easy_transformer 
tokens_dataset = easy_transformer.utils.tokenize_and_concatenate(dataset, model.tokenizer, streaming=False, max_length=model_cfg.n_ctx, column_name="text", add_bos_token=True, num_proc=4)
data_loader = torch.utils.data.DataLoader(tokens_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
print("Number of batches:", len(data_loader))
#test_string_trained = "Hello world this is a test of overfitting"
print("done")

['<|endoftext|>', 'On', ' hall', 'ow', 'een', ',', ' all', ' the', ' children', ' go', ' T', 'rick', ' or']
tensor([[   0, 2374, 7423,  319, 9673,   13,  512,  253, 2151,  564,  308, 4662,
          390]], device='cuda:0')


  0%|          | 0/1 [00:00<?, ?it/s]

done one


ModuleNotFoundError: No module named 'EasyTransformer'

In [ ]:
print("done")

done


In [17]:
# Test Prompt Util -- Check the logit score of the expected output vs. the actual
#                     output
example_prompt = "the founder of Facebook is Mark"
example_answer = "Zuckerberg"

utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)



Tokenized prompt: ['<|endoftext|>', 'the', ' founder', ' of', ' Facebook', ' is', ' Mark']
Tokenized answer: [' Z', 'ucker', 'berg']


Performance on answer token:
Rank: 0        Logit: 22.91 Prob: 99.94% Token: | Z|

Top 0th token. Logit: 22.91 Prob: 99.94% Token: | Z|
Top 1th token. Logit: 15.02 Prob:  0.04% Token: | z|
Top 2th token. Logit: 12.46 Prob:  0.00% Token: | E|
Top 3th token. Logit: 12.16 Prob:  0.00% Token: | Elliot|
Top 4th token. Logit: 11.97 Prob:  0.00% Token: |
|
Top 5th token. Logit: 11.81 Prob:  0.00% Token: | Cuban|
Top 6th token. Logit: 11.52 Prob:  0.00% Token: |Z|
Top 7th token. Logit: 11.40 Prob:  0.00% Token: |  |
Top 8th token. Logit: 10.68 Prob:  0.00% Token: |us|
Top 9th token. Logit: 10.56 Prob:  0.00% Token: |.|


Performance on answer token:
Rank: 0        Logit: 23.77 Prob: 99.60% Token: |ucker|

Top 0th token. Logit: 23.77 Prob: 99.60% Token: |ucker|
Top 1th token. Logit: 17.93 Prob:  0.29% Token: |uk|
Top 2th token. Logit: 16.55 Prob:  0.07% Token: |uck|
Top 3th token. Logit: 15.53 Prob:  0.03% Token: |uc|
Top 4th token. Logit: 13.66 Prob:  0.00% Token: |UCK|
Top 5th token. Logit: 12.91 Prob:  0.00% Token: |.|
Top 6th token. Logit: 11.87 Prob:  0.00% Token: |ub|
Top 7th token. Logit: 11.50 Prob:  0.00% Token: |ucks|
Top 8th token. Logit: 11.00 Prob:  0.00% Token: |ander|
Top 9th token. Logit: 10.94 Prob:  0.00% Token: |im|


Performance on answer token:
Rank: 0        Logit: 25.24 Prob: 98.27% Token: |berg|

Top 0th token. Logit: 25.24 Prob: 98.27% Token: |berg|
Top 1th token. Logit: 21.15 Prob:  1.65% Token: |burg|
Top 2th token. Logit: 17.57 Prob:  0.05% Token: |ber|
Top 3th token. Logit: 16.38 Prob:  0.01% Token: |borg|
Top 4th token. Logit: 15.81 Prob:  0.01% Token: |beg|
Top 5th token. Logit: 13.86 Prob:  0.00% Token: |bert|
Top 6th token. Logit: 13.72 Prob:  0.00% Token: |bur|
Top 7th token. Logit: 13.15 Prob:  0.00% Token: |­|
Top 8th token. Logit: 12.94 Prob:  0.00% Token: |b|
Top 9th token. Logit: 12.89 Prob:  0.00% Token: |berger|


Ranks of the answer tokens: [(' Z', 0), ('ucker', 0), ('berg', 0)]

In [11]:
pprint([(name, param.shape) for name, param in model.named_parameters()])

[('embed.W_E', torch.Size([50280, 4096])),
 ('blocks.0.ln1.w', torch.Size([4096])),
 ('blocks.0.ln1.b', torch.Size([4096])),
 ('blocks.0.ln2.w', torch.Size([4096])),
 ('blocks.0.ln2.b', torch.Size([4096])),
 ('blocks.0.attn.W_Q', torch.Size([32, 4096, 128])),
 ('blocks.0.attn.W_K', torch.Size([32, 4096, 128])),
 ('blocks.0.attn.W_V', torch.Size([32, 4096, 128])),
 ('blocks.0.attn.W_O', torch.Size([32, 128, 4096])),
 ('blocks.0.attn.b_Q', torch.Size([32, 128])),
 ('blocks.0.attn.b_K', torch.Size([32, 128])),
 ('blocks.0.attn.b_V', torch.Size([32, 128])),
 ('blocks.0.attn.b_O', torch.Size([4096])),
 ('blocks.0.mlp.W_in', torch.Size([4096, 16384])),
 ('blocks.0.mlp.b_in', torch.Size([16384])),
 ('blocks.0.mlp.W_out', torch.Size([16384, 4096])),
 ('blocks.0.mlp.b_out', torch.Size([4096])),
 ('blocks.1.ln1.w', torch.Size([4096])),
 ('blocks.1.ln1.b', torch.Size([4096])),
 ('blocks.1.ln2.w', torch.Size([4096])),
 ('blocks.1.ln2.b', torch.Size([4096])),
 ('blocks.1.attn.W_Q', torch.Size([32, 

In [15]:
# Testing out Dolly's Q/A ability.

model.generate(
    "abcdefghi",

    max_new_tokens=100,
)

# This Dolly is kind of dumb.
# We should try to get A100 x 8 Cluster ASAP and get a full sized model.

  0%|          | 0/100 [00:00<?, ?it/s]

'abcdefghi\n1234567890\nabcdefghij\n1234567890\nabcdefghijk\n1234567890\nabcdefghijkl\n1234567890\nabcdefghijklm\n1234567890\nabcdefghijklmn\n1234567890\nabcdefghijklmno\n1234567890\nabcdefghijklmnop\n1234567890\nabcdefghik\n01234567891\nabcdefghikj\n01234567892\n'

In [13]:
# Direct Logit Attribution
